In [ ]:
import PIL.Image as Image
import gradio as gr
from ultralytics import YOLO
from groq import Groq
import os

# YOLO Model Initialization
model = YOLO("/content/best.pt")

# Groq API Initialization
api_key = os.getenv("GROQ_API_KEY", "USE_YOUR_OWN_KEY")
client = Groq(api_key=api_key)

# Knowledge Base for Remedies
remedy_knowledge_base = {
    "elephant": "To clear an elephant, use loud noises like horns or fireworks.",
    "cow": "Guide a cow back using food or by gently leading it.",
    "buffalo": "Buffalo can be deterred with barriers or loud sounds.",
    "snake": "Stay calm and call animal control if you see a snake.",
    "goat": "Offer food to lure a goat or gently call it by name.",
    "dog": "Approach calmly and check for identification on the collar."
}

# YOLO Prediction and Remedy
def predict_image(img, conf_threshold, iou_threshold):
    results = model.predict(
        source=img,
        conf=conf_threshold,
        iou=iou_threshold,
        show_labels=True,
        show_conf=True,
        imgsz=640,
    )

    remedy_text = "No animals detected."
    im = None

    for r in results:
        im_array = r.plot()
        im = Image.fromarray(im_array[..., ::-1])

        detected_classes = [model.names[int(c)] for c in r.boxes.cls]

        # Get remedies if animals are detected
        for animal in detected_classes:
            if animal in remedy_knowledge_base:
                remedy_text = f"{animal.capitalize()} detected. Remedy: {remedy_knowledge_base[animal]}"
                break

    return im, remedy_text

# Chatbot for General Queries (with limited chat history)
chat_history = []  # Store conversation history (last 3 prompts)

def generate_response(user_input, detected_animal=None):
    global chat_history
    try:
        # Format detected animal context
        animal_context = f"Detected animal: {detected_animal}" if detected_animal else ""

        # Append user input (with detected animal context if available)
        chat_history.append({"role": "user", "content": f"{animal_context}\n{user_input}"})

        # Retain only the last 3 exchanges
        chat_history = chat_history[-3:]

        # Search for remedies in knowledge base
        for animal, remedy in remedy_knowledge_base.items():
            if animal.lower() in user_input.lower():
                response = f"Remedy for {animal.capitalize()}: {remedy}"
                chat_history.append({"role": "assistant", "content": response})
                return response

        # Call Groq API with limited chat history
        completion = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=chat_history,
            temperature=1,
            max_tokens=1024,
            top_p=1,
            stream=False
        )

        # Extract and store assistant response
        response = completion.choices[0].message.content
        chat_history.append({"role": "assistant", "content": response})

        return response

    except Exception as e:
        return f"Error with Groq API: {e}"


# Modify YOLO Detection to Pass Animal to Chatbot (with proper outputs)
def predict_image(img, conf_threshold, iou_threshold):
    results = model.predict(
        source=img,
        conf=conf_threshold,
        iou=iou_threshold,
        show_labels=True,
        show_conf=True,
        imgsz=640,
    )

    remedy_text = "No animals detected."
    im = None
    detected_animal = None

    for r in results:
        im_array = r.plot()
        im = Image.fromarray(im_array[..., ::-1])

        detected_classes = [model.names[int(c)] for c in r.boxes.cls]

        for animal in detected_classes:
            if animal in remedy_knowledge_base:
                remedy_text = f"{animal.capitalize()} detected. Remedy: {remedy_knowledge_base[animal]}"
                detected_animal = animal  # Pass detected animal name to chatbot
                break

    return im, remedy_text, detected_animal  # Ensure this is the right return structure


# Combined Gradio Interface
with gr.Blocks() as iface:
    gr.Markdown("# Animal Detection and Remedies Chatbot (with Context)")

    with gr.Row():
        with gr.Column():
            gr.Markdown("### Image Detection and Remedy")
            image_input = gr.Image(type="pil", label="Upload Image")
            conf_slider = gr.Slider(0, 1, value=0.25, label="Confidence Threshold")
            iou_slider = gr.Slider(0, 1, value=0.45, label="IoU Threshold")
            detect_button = gr.Button("Detect Animals")
            image_output = gr.Image(type="pil", label="Detected Image")
            remedy_output = gr.Textbox(label="Detection Result")

        with gr.Column():
            gr.Markdown("### Chatbot for Animal Questions (with Memory)")
            user_input = gr.Textbox(label="Your Message", placeholder="Ask about remedies...")
            chatbot_button = gr.Button("Ask")
            chatbot_output = gr.Textbox(label="Chatbot Response")
            reset_button = gr.Button("Reset Chat History")

    # Interaction Logic
    detect_button.click(
        predict_image,
        inputs=[image_input, conf_slider, iou_slider],
        outputs=[image_output, remedy_output, chatbot_output]  # Ensure outputs are valid components
    )

    chatbot_button.click(
        generate_response,
        inputs=[user_input, remedy_output],  # Pass remedy as context to chatbot
        outputs=[chatbot_output]
    )

    reset_button.click(
        reset_chat,
        outputs=[chatbot_output]
    )

if __name__ == "__main__":
    iface.launch()
